In [ ]:
import torchaudio
import random
from datasets import load_dataset
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from IPython.display import Audio
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
%matplotlib inline 

class OneHot(nn.Module):
    def __init__(self, MU):
        super(OneHot, self).__init__()
        self.MU = MU
        self.ones = torch.sparse.torch.eye(MU).to(device)

    def forward(self, x):
        x = x.to(device)
        batch_size, seq_len = x.size()
        x = x.view(-1)

        x_one_hot = self.ones.index_select(0, x)
        x_one_hot = x_one_hot.view(batch_size, seq_len, self.MU)
        x_one_hot = x_one_hot.transpose(1, 2)
        return x_one_hot

    def __repr__(self):
        return self.__class__.__name__ + "({})".format(self.MU)


class Wavenet(nn.Module):
    def __init__(self, dilation_depth, n_blocks, n_dil_channnels, n_residual_channels, n_skip_channels, n_category,
                 kernel_size, seq_len_segment):
        super(Wavenet, self).__init__()
        self.dilation_depth = dilation_depth
        self.n_blocks = n_blocks
        self.n_dil_channnels = n_dil_channnels
        self.n_residual_channels = n_residual_channels
        self.n_skip_channels = n_skip_channels
        self.n_category = n_category
        self.kernel_size = kernel_size
        self.One_hot = OneHot(n_category)
        self.seq_len_segment = seq_len_segment
        ###Building the model###
        self.dilations = [2 ** i for i in range(dilation_depth)] * n_blocks

        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()
        self.residual_convs = nn.ModuleList()
        self.skip_convs = nn.ModuleList()

        ##creating first channels##
        self.input_convs = nn.Conv1d(in_channels=self.n_category, out_channels=self.n_residual_channels, kernel_size=1)
        ###Creating wavenet blocks stacks###
        for d in self.dilations:
            self.filter_convs.append(
                nn.Conv1d(in_channels=n_residual_channels, out_channels=n_dil_channnels, kernel_size=kernel_size,
                          dilation=d))
            self.gate_convs.append(
                nn.Conv1d(in_channels=n_residual_channels, out_channels=n_dil_channnels, kernel_size=kernel_size,
                          dilation=d))
            self.residual_convs.append(
                nn.Conv1d(in_channels=n_dil_channnels, out_channels=n_residual_channels, kernel_size=1))
            self.skip_convs.append(nn.Conv1d(in_channels=n_dil_channnels, out_channels=n_skip_channels, kernel_size=1))
        ##post convoluions
        self.post_conv1 = nn.Conv1d(in_channels=n_skip_channels, out_channels=n_skip_channels, kernel_size=1)
        self.post_conv2 = nn.Conv1d(in_channels=n_skip_channels, out_channels=n_category, kernel_size=1)
    
    def forward(self, x):
        x = x.to(device)
        x = self.One_hot(x)  # One-hot encoding
        x = self.input_convs(x)  # Input convolution
        skip_con = 0
    
        for i in range(self.dilation_depth * self.n_blocks):
            dilation = self.dilations[i]
            res = x
            filter = self.filter_convs[i](x)
            filter = torch.tanh(filter)
            gate = self.gate_convs[i](x)
            gate = torch.sigmoid(gate)
            x = filter * gate
    
            s = self.skip_convs[i](x)
            if isinstance(skip_con, int):
                skip_con = s
            else:
                skip_con = skip_con[:, :, -s.size(2):] + s
            x = self.residual_convs[i](x)
            x = x + res[:, :, dilation * (self.kernel_size - 1):]
    
        x = torch.relu(skip_con)
        x = torch.relu(self.post_conv1(x))
        x = self.post_conv2(x)
    
        # Adjust output length to match the target length
        if x.size(2) > self.seq_len_segment:
            x = x[:, :, :self.seq_len_segment]  # Trim if necessary
        elif x.size(2) < self.seq_len_segment:
            padding = self.seq_len_segment - x.size(2)
            x = torch.nn.functional.pad(x, (0, padding))  # Pad if necessary
    
        return x

    def generate(self, seed_input, num_samples=100):
        gen_list = seed_input.squeeze().tolist()
        # Ensure seed_input length is sufficient
        assert len(gen_list) >= sum(self.dilations) + 1, "Seed input length too short"

        for _ in range(num_samples):
            # Ensure the length of the seed input is sufficient for the network
            if len(gen_list) < sum(self.dilations) + 1:
                padding_length = sum(self.dilations) + 1 - len(gen_list)
                gen_list = [0] * padding_length + gen_list

            x = Variable(torch.LongTensor(gen_list[-sum(self.dilations) - 1:]))
            x = x.unsqueeze(0).to(device)  # Add batch dimension
            
            with torch.no_grad():
                y = self.forward(x)
                y = y.squeeze(0)  # Remove batch dimension

            # Get the most probable next value
            _, i = y.max(dim=1)
            gen_list.append(i[-1].item())
        print("Generate")
        return gen_list
    
class BirdsetDataset(Dataset):
    def __init__(self, hsn, seq_len_segment, mu):
        self.hsn = hsn
        self.seq_len_segment = seq_len_segment
        self.mu = mu
        self.data_list = []
        for sample in self.hsn:
            data, sr = preprocess(sample)
            max_val = torch.max(data)
            min_val = torch.min(data)
            if max_val > torch.abs(min_val):
                data = data / max_val
            else:
                data = data / torch.abs(min_val)
            self.data_list.append(data)

    def __len__(self):
        return len(self.data_list)


    def __getitem__(self, idx):
        data = self.data_list[idx]
        start = np.random.randint(0, data.shape[1] - self.seq_len_segment)
        ys = data[:, start:start + self.seq_len_segment]
        ys = mulaw_quantize(ys, self.mu)
        ys = ys.squeeze(0)
        return ys.to(device)

def mulaw_quantize(x, qc):
    assert isinstance(x, torch.Tensor), 'mu_law_encoding expects a Tensor'
    mu = qc - 1
    if not x.is_floating_point():
        x = x.to(torch.float)
    mu = torch.tensor(mu, dtype=x.dtype)
    x_mu = torch.sign(x) * torch.log1p(mu * torch.abs(x)) / torch.log1p(mu)
    x_mu = ((x_mu + 1) / 2 * mu + 0.5).to(torch.int64)
    return x_mu



def inv_mulaw_quantize(x_mu, quantization_channels=256, device=device):
    mu = quantization_channels - 1.
    if isinstance(x_mu, np.ndarray):
        x = ((x_mu) / mu) * 2 - 1.
        x = np.sign(x) * (np.exp(np.abs(x) * np.log1p(mu)) - 1.) / mu
    elif isinstance(x_mu, (torch.Tensor, torch.LongTensor)):
        if isinstance(x_mu, (torch.LongTensor, torch.cuda.LongTensor)):
            x_mu = x_mu.float()
        if device == "cuda":
            mu = (torch.FloatTensor([mu]))
        else:
            mu = torch.FloatTensor([mu])
        x = ((x_mu) / mu) * 2 - 1.
        x = torch.sign(x) * (torch.exp(torch.abs(x) * torch.log1p(mu)) - 1.) / mu
    return x


# TODO: 
# 
def data_generation(data, fr, seq_len_segment, mu, device=device):
    max = torch.max(data)
    min = torch.min(data)
    comparison = max > torch.abs(min)
    if torch.all(comparison):
        data = torch.div(data, max)
    else:
        # abs_min_vals = torch.abs(min)
        data = torch.div(data, abs(min))
    while True:
        start = np.random.randint(0, data.shape[1] - seq_len_segment)
        ys = data[:, start:start + seq_len_segment]
        ys = mulaw_quantize(ys, mu)
        ys = ys.squeeze(0)
        yield ys.to(device)

def preprocess(batch):
    audio, _ = torchaudio.load(batch['filepath'])
    if audio.shape[0] > 1:
        audio = audio[0] 
    if len(audio.shape) > 1:
        audio = audio.mean(dim=0) 
    return audio.unsqueeze(0), _ 


hsn = load_dataset('DBD-research-group/BirdSet', 'HSN')
subset_percentage = 0.25
subset_indices = random.sample(range(len(hsn['train'])), int(len(hsn['train']) * subset_percentage))
hsn = hsn['train'].select(subset_indices)
seq_len_segment = 40000
mu = 128
batch_size = 8
dataset = BirdsetDataset(hsn, seq_len_segment, mu)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model
dilation_depth = 10
n_blocks = 2
n_dilation_channels = 24
n_residual_channels = 24
n_skip_channels = 128
n_category = mu
kernel_size = 2
model = Wavenet(dilation_depth, n_blocks, n_dilation_channels, n_residual_channels, n_skip_channels, n_category,
                kernel_size, seq_len_segment=seq_len_segment)

model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss since it's a classification problem
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 45  # Adjust based on your needs
model.train()
for epoch in range(epochs):
    for i, inputs in enumerate(dataloader):
        inputs = inputs.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Ensure outputs are of shape (batch_size, num_classes, seq_len)
        outputs = outputs.permute(0, 2, 1)  # Change to (batch_size, seq_len, num_classes)
        
        # Flatten for loss calculation
        batch_size, seq_len, num_classes = outputs.size()
        outputs = outputs.contiguous().view(-1, num_classes)
        targets = inputs.contiguous().view(-1)
        
        # Ensure targets are of type Long
        targets = targets.long()
        
        # Compute loss
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {loss.item() / 10:.4f}')


print('Finished Training')
torch.save(model.state_dict(), 'wavenet_model.pth')
#import soundfile as sf
#output_file = 'output_audio.wav'
#sf.write(output_file, y, sr)


C:\Users\shale\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for DBD-research-group/BirdSet contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/DBD-research-group/BirdSet
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [22]:
import torch
import soundfile as sf
from IPython.display import Audio

# Load your trained model
device = "cuda" if torch.cuda.is_available() else "cpu"
seq_len_segment = 20000
mu = 128
batch_size = 8
dilation_depth = 10
n_blocks = 2
n_dilation_channels = 24
n_residual_channels = 24
n_skip_channels = 128
n_category = mu
kernel_size = 2
model = Wavenet(dilation_depth, n_blocks, n_dilation_channels, n_residual_channels, n_skip_channels, n_category,
                kernel_size, seq_len_segment=seq_len_segment)

model.to(device)

model.load_state_dict(torch.load('wavenet_model.pth'))
model.to(device)
model.eval()
# Function to generate audio
def generate_audio(model, seed_length, num_samples, mu, device):
    # Create a seed input
    seed_input = torch.randint(0, mu, (1, seed_length)).to(device)
    
    # Generate audio sequence
    generated_sequence = model.generate(seed_input, num_samples=num_samples)
    
    # Convert generated sequence to waveform
    generated_sequence_tensor = torch.LongTensor(generated_sequence).to(device)
    waveform = inv_mulaw_quantize(generated_sequence_tensor, quantization_channels=mu, device=device)
    waveform = waveform.cpu().numpy()
    
    return waveform

# Parameters
seed_length = 1000000 
num_samples = 4
mu = 128 

# Generate audio
waveform = generate_audio(model, seed_length, num_samples, mu, device)

# Save the generated audio to a file
sf.write('generated_audio.wav', waveform, 16000) 

# Optionally, play the generated audio
Audio(waveform, rate=16000) 



Generate
